# Google Colab Setup

In [1]:
!git clone https://github.com/haidarihza/sistem-penilaian-kompetensi-v2

!pip install batchbald_redux

import sys
sys.path.append('/content/sistem-penilaian-kompetensi-v2/Penilaian Kompetensi Inggris')

Cloning into 'sistem-penilaian-kompetensi-v2'...
remote: Enumerating objects: 19188, done.
remote: Counting objects: 100% (1060/1060), done.
remote: Compressing objects: 100% (367/367), done.
remote: Total 19188 (delta 666), reused 1060 (delta 666), pack-reused 18128
Receiving objects: 100% (19188/19188), 163.76 MiB | 12.35 MiB/s, done.
Resolving deltas: 100% (2463/2463), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 9.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvi

In [2]:
from google.colab import files
uploaded = files.upload()

Saving dataset.json to dataset.json


In [3]:
import json
for key in uploaded.keys():
  raw_data = json.loads(uploaded[key])['data']

# Evaluation Setup

In [4]:
from src.model import *
import torch

In [5]:
device = torch.device('cuda')

In [6]:
# Set seed
torch.manual_seed(0)

# Active Learning Evaluation

Data split: 7 pool sets / 2 eval sets (1 set = 4 data points)<br>
Base model: cross-encoder/nli-deberta-v3-base<br>

Acquisition function: PowerBALD (α=5)<br>
Acquisition size: 10<br>
Bayesian sample size: 5<br>
Acquisition step: 2<br>

In [7]:
pool_data = raw_data[:7]
eval_data = raw_data[7:]

transcripts = [transcript for d in pool_data for transcript in d['transcripts']]
competence_sets = [d['competence_levels'] for d in pool_data for _ in range(len(d['transcripts']))]
labels = [label for d in pool_data for label in d['labels']]
pool_dataset = TCDataset(transcripts, competence_sets, labels)

transcripts = [transcript for d in eval_data for transcript in d['transcripts']]
competence_sets = [d['competence_levels'] for d in eval_data for _ in range(len(d['transcripts']))]
labels = [label for d in eval_data for label in d['labels']]
eval_dataset = TCDataset(transcripts, competence_sets, labels)

In [8]:
model = CompetenceModel.load('cross-encoder/nli-deberta-v3-base', 'crossencoder', state_dict_path=None, device=device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/417 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/18.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:562: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/738M [00:00<?, ?B/s]

In [9]:
# Model prediction on eval dataset (initial)

with torch.no_grad():
    scores = model(eval_dataset.transcripts, eval_dataset.competence_sets)
    acc = torch.sum(torch.argmax(scores, dim=1) == torch.tensor(eval_dataset.label_indices, device=device)).item() / scores.size(0)

print('Predict probabilities:')
print(scores)
print(f'Accuracy: {acc:.2f}')

Predict probabilities:
tensor([[8.8539e-01, 2.4814e-03, 2.4600e-03, 1.0966e-01],
        [3.1869e-01, 5.6854e-02, 2.3564e-01, 3.8882e-01],
        [9.1472e-02, 7.9833e-02, 2.5287e-01, 5.7583e-01],
        [1.7241e-03, 1.1876e-03, 1.6624e-03, 9.9543e-01],
        [7.4673e-01, 2.5297e-01, 1.4159e-04, 1.6520e-04],
        [3.3627e-03, 9.5070e-01, 4.5272e-02, 6.6608e-04],
        [5.4032e-02, 1.3424e-01, 8.0497e-01, 6.7584e-03],
        [1.4482e-01, 7.0768e-01, 9.1833e-02, 5.5661e-02]], device='cuda:0')
Accuracy: 0.62


## Acquisition Step 1

In [10]:
bayesian_model = BayesianCompetenceModel(model)

In [11]:
with torch.no_grad():
    # Split to minimize GPU RAM usage
    scores = []
    n = len(pool_dataset)
    for i in range(0, n, 5):
      score = bayesian_model(pool_dataset.transcripts[i:min(i+5, n)], pool_dataset.competence_sets[i:min(i+5, n)], k=5)
      scores.append(score)

    scores = torch.cat(scores)
    log_scores = torch.log(scores)
    batch = get_powerbald_batch(log_scores, batch_size=10, alpha=5)

Entropy:   0%|          | 0/28 [00:00<?, ?it/s]

Conditional Entropy:   0%|          | 0/28 [00:00<?, ?it/s]

In [12]:
train_dataset = torch.utils.data.Subset(pool_dataset, batch.indices)

transcripts = [t for i, t in enumerate(pool_dataset.transcripts) if i not in batch.indices]
competence_sets = [c for i, c in enumerate(pool_dataset.competence_sets) if i not in batch.indices]
labels = [l for i, l in enumerate(pool_dataset.label_indices) if i not in batch.indices]
pool_dataset = TCDataset(transcripts, competence_sets, labels)

In [13]:
# Model training on 5 epochs

model = CompetenceModel.load('cross-encoder/nli-deberta-v3-base', 'crossencoder', state_dict_path=None, device=device)

model.fit(train_dataset, eval_dataset, epochs=5, batch_size=4, early_stop=False,
          optimizer_cls=torch.optim.Adam, optimizer_params={'lr': 1e-5})

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:562: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Epoch 1 | Train Loss = 2.5892 | Train Acc = 0.2000 | Val Loss = 0.9257 | Val Acc = 0.5000
Epoch 2 | Train Loss = 1.0633 | Train Acc = 0.6000 | Val Loss = 0.9555 | Val Acc = 0.5000
Epoch 3 | Train Loss = 0.9525 | Train Acc = 0.7000 | Val Loss = 0.9841 | Val Acc = 0.5000
Epoch 4 | Train Loss = 0.7328 | Train Acc = 0.9000 | Val Loss = 1.0172 | Val Acc = 0.5000
Epoch 5 | Train Loss = 0.7457 | Train Acc = 0.8000 | Val Loss = 1.0389 | Val Acc = 0.5000


In [14]:
# Model prediction on eval dataset

with torch.no_grad():
    scores = model(eval_dataset.transcripts, eval_dataset.competence_sets)
    acc = torch.sum(torch.argmax(scores, dim=1) == torch.tensor(eval_dataset.label_indices, device=device)).item() / scores.size(0)

print('Predict probabilities:')
print(scores)
print(f'Accuracy: {acc:.2f}')

Predict probabilities:
tensor([[6.5911e-01, 1.4635e-02, 1.7072e-02, 3.0918e-01],
        [2.5308e-01, 7.9172e-02, 2.5758e-01, 4.1017e-01],
        [1.0301e-01, 6.1185e-02, 2.8674e-01, 5.4907e-01],
        [4.4123e-03, 2.7963e-03, 4.8472e-03, 9.8794e-01],
        [9.9756e-01, 1.7274e-03, 3.4646e-04, 3.6517e-04],
        [8.9488e-02, 8.7467e-01, 2.5620e-02, 1.0226e-02],
        [3.9485e-01, 3.0714e-01, 1.8607e-01, 1.1194e-01],
        [1.9747e-01, 5.0298e-01, 1.9712e-01, 1.0243e-01]], device='cuda:0')
Accuracy: 0.50


## Acquisition Step 2

In [15]:
bayesian_model = BayesianCompetenceModel(model)

In [16]:
with torch.no_grad():
    # Split to minimize GPU RAM usage
    scores = []
    n = len(pool_dataset)
    for i in range(0, n, 5):
      score = bayesian_model(pool_dataset.transcripts[i:min(i+5, n)], pool_dataset.competence_sets[i:min(i+5, n)], k=5)
      scores.append(score)

    scores = torch.cat(scores)
    log_scores = torch.log(scores)
    batch = get_powerbald_batch(log_scores, batch_size=10, alpha=5)

Entropy:   0%|          | 0/18 [00:00<?, ?it/s]

Conditional Entropy:   0%|          | 0/18 [00:00<?, ?it/s]

In [17]:
train_dataset = torch.utils.data.ConcatDataset([train_dataset, torch.utils.data.Subset(pool_dataset, batch.indices)])

transcripts = [t for i, t in enumerate(pool_dataset.transcripts) if i not in batch.indices]
competence_sets = [c for i, c in enumerate(pool_dataset.competence_sets) if i not in batch.indices]
labels = [l for i, l in enumerate(pool_dataset.label_indices) if i not in batch.indices]
pool_dataset = TCDataset(transcripts, competence_sets, labels)

In [19]:
# Model training on 5 epochs

model = CompetenceModel.load('cross-encoder/nli-deberta-v3-base', 'crossencoder', state_dict_path=None, device=device)

model.fit(train_dataset, eval_dataset, epochs=5, batch_size=4, early_stop=False,
          optimizer_cls=torch.optim.Adam, optimizer_params={'lr': 1e-5})

Epoch 1 | Train Loss = 1.3968 | Train Acc = 0.4500 | Val Loss = 0.9916 | Val Acc = 0.5000
Epoch 2 | Train Loss = 1.1977 | Train Acc = 0.4500 | Val Loss = 1.0795 | Val Acc = 0.5000
Epoch 3 | Train Loss = 1.0478 | Train Acc = 0.6000 | Val Loss = 1.0803 | Val Acc = 0.5000
Epoch 4 | Train Loss = 0.9259 | Train Acc = 0.6000 | Val Loss = 1.0310 | Val Acc = 0.6250
Epoch 5 | Train Loss = 0.7610 | Train Acc = 0.7000 | Val Loss = 1.0496 | Val Acc = 0.6250


In [20]:
# Model prediction on eval dataset after training

with torch.no_grad():
    scores = model(eval_dataset.transcripts, eval_dataset.competence_sets)
    acc = torch.sum(torch.argmax(scores, dim=1) == torch.tensor(eval_dataset.label_indices, device=device)).item() / scores.size(0)

print('Predict probabilities:')
print(scores)
print(f'Accuracy: {acc:.2f}')

Predict probabilities:
tensor([[5.8807e-01, 2.3597e-02, 2.7302e-02, 3.6103e-01],
        [4.8530e-01, 8.1891e-02, 2.3930e-01, 1.9350e-01],
        [3.9478e-02, 1.8958e-02, 7.1795e-01, 2.2362e-01],
        [4.3310e-03, 3.1921e-03, 1.1751e-02, 9.8073e-01],
        [9.8973e-01, 9.7046e-03, 2.2674e-04, 3.3922e-04],
        [1.0754e-01, 8.8325e-01, 5.3105e-03, 3.8957e-03],
        [3.4063e-01, 3.9117e-01, 9.6778e-02, 1.7142e-01],
        [1.5157e-01, 6.8259e-01, 8.7218e-02, 7.8619e-02]], device='cuda:0')
Accuracy: 0.62
